In [27]:
import pandas as pd
import tensorflow as tf
from pandas import DataFrame
import numpy as np
import math
import heapq
from tqdm import tqdm
import random
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#!pip install --upgrade tensorflow
#!pip install --upgrade pandas
#!pip install --upgrade numpy

Num GPUs Available:  1


In [28]:
df = pd.read_csv('users-feeds.csv')

def shrink_users_df(df,user_id):
    userIds = np.random.choice(df[user_id].unique(),
                                    size=int(len(df[user_id].unique())*0.10),
                                    replace=False)
    return df.loc[df[user_id].isin(userIds)]
def add_negative_samples(df, item_tag, user_tag,label_tag):

    updated_df = pd.DataFrame(columns=[user_tag,item_tag,label_tag])
    all_feeds = df[item_tag].unique()
    users, items, labels = [], [], []

    user_item_set = set(zip(df[user_tag], df[item_tag]))
    num_negatives = 2

    for (u, i) in user_item_set:
        users.append(u)
        items.append(i)
        labels.append(1) # items that the user has interacted with are positive
        for _ in range(num_negatives):
            # randomly select an item
            negative_item = np.random.choice(all_feeds)
            # check that the user has not interacted with this item
            while (u, negative_item) in user_item_set:
                negative_item = np.random.choice(all_feeds)
            users.append(u)
            items.append(negative_item)
            labels.append(0) # items not interacted with are negative
    updated_df[user_tag] = users
    updated_df[item_tag] = items
    updated_df[label_tag] = labels
    del df
    return updated_df

def mask_first(x):
    """
    Return a list of 0 for the first item and 1 for all others
    """
    result = np.ones_like(x)
    result[0] = 0

    return result

# needs to add validate in the future
def train_test_split(full_df):
    df_test = df.copy(deep=True)
    df_test = df_test.groupby(['user']).first()

    df_test['user'] = df_test.index
    df_test = df_test[['user', 'feed_id','is_following_feed']]
    df_test = df_test.rename_axis(None, axis=1)

    df_train = df.copy(deep=True)
    mask = df.groupby(['user'])['user'].transform(mask_first).astype(bool)

    df_train = df.loc[mask]
    return df_train, df_test


In [29]:
df = shrink_users_df(df, 'user')


df.loc[:, 'is_following_feed'] = 1

df = add_negative_samples(df,'feed_id','user','is_following_feed')

print(df.sample(10))

df_train, df_test = train_test_split(df)


           user  feed_id  is_following_feed
1125867   18145     1754                  1
3924527     747  2988973                  0
3797266   45212  6656174                  0
718425   192108  3631556                  1
3544823    2562  7728650                  0
1879120   91278  3251087                  0
244217    12323  5977891                  0
3424530  182926   453441                  1
1063721  159008  6160481                  0
3733494  335385  6679019                  1


In [30]:
def _get_user_embedding_layers(inputs, emb_dim):  
    """ create user embeddings """  
    user_gmf_emb = tf.keras.layers.Dense(emb_dim, activation='relu')(inputs)  
    user_mlp_emb = tf.keras.layers.Dense(emb_dim, activation='relu')(inputs)  
    return user_gmf_emb, user_mlp_emb  
  
def _get_item_embedding_layers(inputs, emb_dim):  
    """ create item embeddings """  
    item_gmf_emb = tf.keras.layers.Dense(emb_dim, activation='relu')(inputs)  
    item_mlp_emb = tf.keras.layers.Dense(emb_dim, activation='relu')(inputs)  
    return item_gmf_emb, item_mlp_emb  

In [31]:
def _gmf(user_emb, item_emb):  
    """ general matrix factorization branch """  
    gmf_mat = tf.keras.layers.Multiply()([user_emb, item_emb])  
    return gmf_mat  

In [32]:
def _mlp(user_emb, item_emb, dropout_rate):  
    """ multi-layer perceptron branch """  
    def add_layer(dim, input_layer, dropout_rate):  
        hidden_layer = tf.keras.layers.Dense(dim, activation='relu')(input_layer)  
        if dropout_rate:  
            dropout_layer = tf.keras.layers.Dropout(dropout_rate)(hidden_layer)  
            return dropout_layer  
        return hidden_layer  
  
    concat_layer = tf.keras.layers.Concatenate()([user_emb, item_emb])  
    dropout_l1 = tf.keras.layers.Dropout(dropout_rate)(concat_layer)  
    dense_layer_1 = add_layer(64, dropout_l1, dropout_rate)  
    dense_layer_2 = add_layer(32, dense_layer_1, dropout_rate)  
    dense_layer_3 = add_layer(16, dense_layer_2, None)  
    dense_layer_4 = add_layer(8, dense_layer_3, None)  
    return dense_layer_4  

In [33]:
def _neuCF(gmf, mlp, dropout_rate):  
    """ final output layer """  
    concat_layer = tf.keras.layers.Concatenate()([gmf, mlp])  
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(concat_layer)  
    return output_layer  

In [34]:
def batch_generator(x, y, batch_size, n_batch, shuffle, user_dim, item_dim):
    """ batch generator to supply data for training and testing """

    user_df, item_df = x

    counter = 0
    training_index = np.arange(user_df.shape[0])

    if shuffle:
        np.random.shuffle(training_index)

    while True:
        batch_index = training_index[batch_size*counter:batch_size*(counter+1)]
        user_batch = tf.one_hot(user_df[batch_index], depth=user_dim)
        item_batch = tf.one_hot(item_df[batch_index], depth=item_dim)
        y_batch = y[batch_index]
        counter += 1
        yield [user_batch, item_batch], y_batch

        if counter == n_batch:
            if shuffle:
                np.random.shuffle(training_index)
            counter = 0

In [35]:
def build_graph(user_dim, item_dim, dropout_rate=0.25):
    """ neural collaborative filtering model """

    user_input = tf.keras.Input(shape=(user_dim))
    item_input = tf.keras.Input(shape=(item_dim))

    # create embedding layers
    user_gmf_emb, user_mlp_emb = _get_user_embedding_layers(user_input, 32)
    item_gmf_emb, item_mlp_emb = _get_item_embedding_layers(item_input, 32)

    # general matrix factorization
    gmf = _gmf(user_gmf_emb, item_gmf_emb)

    # multi layer perceptron
    mlp = _mlp(user_mlp_emb, item_mlp_emb, dropout_rate)

    # output
    output = _neuCF(gmf, mlp, dropout_rate)

    # create the model
    model = tf.keras.Model(inputs=[user_input, item_input], outputs=output)

    return model

In [ ]:
def eval_hit_rate(test_df, full_df, model):
  test_user_item_set = set(zip(test_df['user'], test_df['feed_id']))

  user_interacted_items = full_df.groupby('user')['feed_id'].apply(list).to_dict()

  hits = []
  for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(full_df['feed_id'].unique()) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:5].tolist()]

    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))



In [36]:
def model(x_train, y_train, n_user, n_item, num_epoch, batch_size):

    num_batch = np.ceil(x_train[0].shape[0]/batch_size)

    # build graph
    model = build_graph(n_user, n_item)

    # compile and train
    optimizer = tf.keras.optimizers.Adam(learning_rate=.002)

    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy']
                  )

    model.fit_generator(
        generator=batch_generator(
            x=x_train, y=y_train,
            batch_size=batch_size, n_batch=num_batch,
            shuffle=True, user_dim=n_user, item_dim=n_item),
        epochs=num_epoch,
        steps_per_epoch=num_batch,
        verbose=2
    )

    return model

In [37]:
df['user'].to_numpy()

array([289879, 289879, 289879, ..., 201877, 201877, 201877])

In [ ]:


ncf_model = model(
        x_train=[df_train['user'].to_numpy(), df_train['feed_id'].to_numpy()],
        y_train=df_train['is_following_feed'].to_numpy(),
        n_user=df_train['user'].nunique(),
        n_item=df_train['feed_id'].nunique(),
        num_epoch=5,
        batch_size=512
    )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
8306/8306 - 7012s - loss: 0.5619
Epoch 2/5
